### Epic Level Email Notification Automation:

#### Contains obsolete JQL Query and cannot be used
#### Only to be used when the number of Epics are equal to or greater than 100, otherwise not

In [38]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

# JIRA API credentials
base_url = "https://corecard.atlassian.net/"
api_token = ""
email = ""

# API endpoint to fetch issues
endpoint = f"{base_url}rest/api/3/search"
headers = {
    "Accept": "application/json"
}

# JQL query
jql = "project = CCP AND issuetype = epic AND fixVersion in('R40','R39') AND status not in (Cancelled)"
params = {
    "jql": jql,
    "maxResults": 100, 
    "startAt": 0        # Starting index for pagination
}

# List to store all issues
all_issues = []

try:
    while True:
        # Request
        response = requests.get(
            endpoint,
            headers=headers,
            params=params,
            auth=HTTPBasicAuth(email, api_token)
        )

        if response.status_code == 200:
            data = response.json()

            # Extracting issues from the response
            issues = data.get('issues', [])
            all_issues.extend(issues)

            # Checking if there are more issues to fetch
            total = data.get('total', 0)
            start_at = params['startAt']
            max_results = params['maxResults']
            
            if start_at + max_results >= total:
                break  # All issues have been fetched

            # Update startAt to fetch the next page
            params['startAt'] += max_results
        
            # Prepare data for DataFrame
            df_data = []
            for issue in all_issues:
                key = issue.get('key', '')
                fields = issue.get('fields', {})

            # Iterating over each issue and extracting necessary information from the JSON file
            for issue in issues:
                key = issue.get('key', '')
                fields = issue.get('fields', {})

                # Extracting the required fields
                summary = fields.get('summary', '')
                status = fields.get('status', {})
                status_name = status.get('name', '') if status else ''
                assignee = fields.get('assignee', {})
                assignee_name = assignee.get('displayName', 'Unassigned') if assignee else 'Unassigned'
                created = fields.get('created', '-')
                fix_versions = fields.get('fixVersions', [])
                fix_versions_names = [version.get('name', '') for version in fix_versions]
                fix_versions_str = ', '.join(fix_versions_names)
                parent = fields.get('parent', {})
                parent_key = parent.get('key', '-')
                # Custom fields
                ba = fields.get('customfield_10112', [])
                ba_manager = ba[0].get('displayName') if ba else None
                ba_res = fields.get('customfield_10113', [])
                ba_resource = ba_res[0].get('displayName') if ba_res else None
                dev = fields.get('customfield_10114', [])
                dev_manager = dev[0].get('displayName') if dev else None
                dev_resource = fields.get('customfield_10115', [])
                development_resource = dev_resource[0].get('displayName') if dev_resource else None
                t_manager = fields.get('customfield_10116', [])
                test_manager = t_manager[0].get('displayName') if t_manager else None
                t_resource = fields.get('customfield_10117', [])
                test_resource = t_resource[0].get('displayName') if t_resource else None
                u_manager = fields.get('customfield_10118', [])
                uat_manager = u_manager[0].get('displayName') if u_manager else None
                u_resource = fields.get('customfield_10119', [])
                uat_resource = u_resource[0].get('displayName') if u_resource else None
                baeffort = fields.get('customfield_10122', None)
                ba_effort = baeffort
                qaeffort = fields.get('customfield_10125', None)
                qa_effort = qaeffort

                # Appending the extracted information as a dictionary to the list
                df_data.append({
                    'Key': key,
                    'Summary': summary,
                    'Status': status_name,
                    'Assignee': assignee_name,
                    'Created': created,
                    'Fix Version': fix_versions_str,
                    'Parent Key': parent_key,
                    'BA Manager': ba_manager,
                    'BA Resource': ba_resource,
                    'Dev Manager': dev_manager,
                    'Dev Resource': development_resource,
                    'Test Manager': test_manager,
                    'Test Resource': test_resource,
                    'UAT Manager': uat_manager,
                    'UAT Resource': uat_resource,
                    'BA Effort': ba_effort,
                    'QA Effort': qa_effort
                })

            # Creating a Master DataFrame from the extracted data
            df = pd.DataFrame(df_data)

            # Function to make the Key column clickable
            def make_key_clickable(df):
                df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')
                return df

            # Method to process the DataFrame based on field_id:
                # It takes two arguments, the dataframe to bifurcate and the ID of the custom field
                # Then it returns a dataframe for the relevant field_id
            def segregate_dataframe(df, field_id):
                #each if condition returns a dataframe (df_filtered) that has either of the 'N' column empty, replaces 'None' or 'NaN' with '-' 
                if field_id == 'customfield_10112':  # BA Manager field_id
                    df_filtered = df[df[['BA Manager', 'BA Resource', 'BA Effort']].isna().any(axis=1)]
                    df_filtered = make_key_clickable(df_filtered)
                    df_filtered = df_filtered.fillna('-') 
                    return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Fix Version', 'BA Manager', 'BA Resource', 'BA Effort']]
                elif field_id == 'customfield_10114':  # Dev Manager field_id
                    df_filtered = df[df[['Dev Manager', 'Dev Resource']].isna().any(axis=1)]
                    df_filtered = make_key_clickable(df_filtered)
                    df_filtered = df_filtered.fillna('-')
                    return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Fix Version', 'Dev Manager', 'Dev Resource']]
                elif field_id == 'customfield_10118':  # UAT Manager field_id
                    df_filtered = df[df[['UAT Manager', 'UAT Resource']].isna().any(axis=1)]
                    df_filtered = make_key_clickable(df_filtered)
                    df_filtered = df_filtered.fillna('-')
                    return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Fix Version', 'UAT Manager', 'UAT Resource']]
                elif field_id == 'customfield_10116':  # Test Manager field_id
                    df_filtered = df[df[['Test Manager', 'Test Resource']].isna().any(axis=1)]
                    df_filtered = make_key_clickable(df_filtered)
                    df_filtered = df_filtered.fillna('-')
                    return df_filtered[['Key', 'Summary', 'Status', 'Assignee', 'Fix Version', 'Test Manager', 'Test Resource', 'QA Effort']]
                else:
                    raise ValueError("Invalid field_id provided")

            # DataFrames for both BA Manager and Dev Manager
            df_ba_manager = segregate_dataframe(df, 'customfield_10112')
            df_dev_manager = segregate_dataframe(df, 'customfield_10114')
            df_uat_manager = segregate_dataframe(df, 'customfield_10118')
            df_test_manager = segregate_dataframe(df, 'customfield_10116')

            # A function for html styling
            def style_table(html):
                return f"""
                <html>
                <head>
                <style>
                table {{
                    width: 100%;
                    border-collapse: collapse;
                }}
                th, td {{
                    border: 1px solid #ddd;
                    padding: 8px;
                    text-align: center;  
                    white-space: nowrap; 
                    max-width: 200px;  
                    overflow: hidden; 
                    text-overflow: ellipsis;  
                }}
                tr:nth-child(even) {{
                    background-color: #f2f2f2;
                }}
                th {{
                    background-color: #4CAF50;
                    color: white;
                    text-align: center;  
                    white-space: nowrap;
                    max-width: 200px;  
                }}
                </style>
                </head>
                <body>
                <p>Please fill out the required data in the table below:</p>
                {html}
                <p>Sanidhya Mitra</p> 
                <p>PMO Intern at CoreCard</p>
                <p>CoreCard India Software Pvt Ltd</p>
                </body>
                </html>
                """
            
            # Converting DataFrames to HTML tables
            df_ba_manager_html = style_table(df_ba_manager.to_html(index=False, escape=False))
            df_dev_manager_html = style_table(df_dev_manager.to_html(index=False, escape=False))
            df_uat_manager_html = style_table(df_uat_manager.to_html(index=False, escape=False))
            df_test_manager_html = style_table(df_test_manager.to_html(index=False, escape=False))

            # Email configurations
            smtp_server = "corecard-com.mail.protection.outlook.com"
            smtp_port = 25
            smtp_user = 'sanidhya.mitra@corecard.com'

            # Recipients for BA and Dev teams along with the cc
                # Add as many emails as needed, but they should be in the relevant array
            ba_recipients = [
                # 'sanidhya.mitra@corecard.com',
                # 'kalyani.kharate@corecard.com',
                # 'manmohan.singh@corecard.com',
                # 'pawan.linjhara@corecard.com'
            ]
            
            dev_recipients = [
                # 'sanidhya.mitra@corecard.com',
                # 'kalyani.kharate@corecard.com',
                # 'manmohan.singh@corecard.com',
                # 'pawan.linjhara@corecard.com'
            ]

            uat_recipients = [
                # 'sanidhya.mitra@corecard.com',
                # 'kalyani.kharate@corecard.com',
                # 'manmohan.singh@corecard.com',
                # 'pawan.linjhara@corecard.com'
            ]

            testing_recipients = [
                # 'sanidhya.mitra@corecard.com',
                # 'kalyani.kharate@corecard.com',
                # 'manmohan.singh@corecard.com',
                # 'pawan.linjhara@corecard.com'
            ]
            
            cc_email = "sanidhya.mitra@corecard.com"

            # Function to send email
            def send_email(subject, body, to_emails, cc_emails):
                msg = MIMEMultipart()
                msg['From'] = smtp_user
                msg['To'] = ', '.join(to_emails)
                msg['Cc'] = ', '.join(cc_emails)
                msg['Subject'] = subject

                # Attaching the main message body:
                msg.attach(MIMEText(body, 'html'))

                try:
                    with smtplib.SMTP(smtp_server, smtp_port) as server:
                        server.starttls()
                        server.sendmail(smtp_user, to_emails + cc_emails, msg.as_string())
                        print(f"Email sent to {', '.join(to_emails)} with CC to {', '.join(cc_emails)}")
                except Exception as e:
                    print(f"Failed to send email. Error: {e}")

            # Sending email to BA team
            print('Sending BA Report...')
            send_email("BA Manager Report", df_ba_manager_html, ba_recipients, [cc_email])
            print('Successfully sent email to BA team!')

            # Sending email to Dev team
            print('Sending Development Report...')
            send_email("Dev Manager Report", df_dev_manager_html, dev_recipients, [cc_email])
            print("Sucessfully sent email to Development team!")

            # Sending email to UAT team
            print('Sending UAT Report...')
            send_email("UAT Manager Report", df_uat_manager_html, uat_recipients, [cc_email])
            print('Successfully sent email to UAT team!')

            # Sending email to Testing team
            print('Sending Testing Report...')
            send_email("Test Manager Report", df_test_manager_html, testing_recipients, [cc_email])
            print('Successfully sent email to Testing team!')
        else:
            print(f"Failed to fetch data from JIRA API. Status Code: {response.status_code}")
            print(f"Response: {response.text}")

except Exception as e:
    print(f"An error occurred: {e}")


C:\Users\Sanidhya.Mitra\AppData\Local\Temp\ipykernel_12388\4038383372.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')
C:\Users\Sanidhya.Mitra\AppData\Local\Temp\ipykernel_12388\4038383372.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key'] = df['Key'].apply(lambda x: f'<a href="{base_url}browse/{x}" target="_blank">{x}</a>')
C:\Users\Sanidhya.Mitra\AppData\Local\Temp\ipykernel_12388\4038383372.

Sending BA Report...
Email sent to  with CC to sanidhya.mitra@corecard.com
Successfully sent email to BA team!
Sending Development Report...
Email sent to  with CC to sanidhya.mitra@corecard.com
Sucessfully sent email to Development team!
Sending UAT Report...
Email sent to  with CC to sanidhya.mitra@corecard.com
Successfully sent email to UAT team!
Sending Testing Report...
Email sent to  with CC to sanidhya.mitra@corecard.com
Successfully sent email to Testing team!
